**NOTE: This is experimental code, not intended as an example.**

Verilog RTL code and some block diagram code (AXI3 to AXI4-lite conversion) are in the `fpga/prj/axi4lite` directory.

The FPGA image contains a single slave device connected to GPIO and LED pins.
The slave system bus was recoded from the previous undocumented bus to a standard *AXI4-Lite* interface.
The clock for the system bus is derived from the ADC reference oscilator,
as it should be with hardware synchronous to 125 MHz ADC/DAC.

The Example provides raw Python code for mapping registers, without and C code.
And would allow developers to write drivers for FPGA HW entirely in Python.

The first code segment provides register set definition and memory mapping.
Output, output enable and input registers are all 32bit.
GPIOs are mapped to bits `[15:0]`and LEDs are mapped to bits `[23:16]`.

In [ ]:
import os

# copy FPGA bitstream into the firmware search path
# https://www.kernel.org/doc/Documentation/firmware_class/README
os.system("mkdir /lib/firmware/redpitaya")
os.system("python fpga-bit-to-bin.py -f /opt/redpitaya/fpga/axi4lite/fpga.bit /lib/firmware/redpitaya/axi4lite.bin")
# compile device tree overlay
os.system("dtc -I dts -O dtb -o axi4lite.dtbo -@ axi4lite.dts")
# enable kernel debug code
os.system("echo 'file drivers/fpga/* +p' > /sys/kernel/debug/dynamic_debug/control")
# create configfs node
os.system("mkdir /sys/kernel/config/device-tree/overlays/axi4lite")
os.system("cat axi4lite.dtbo > /sys/kernel/config/device-tree/overlays/axi4lite/dtbo")

In [ ]:
import os
import mmap
import numpy as np

regset_gpio = np.dtype([
    ('o', 'uint32'),
    ('t', 'uint32'),
    ('i', 'uint32')
])

#os.system('cat /opt/redpitaya/fpga/axi4lite/fpga.bit > /dev/xdevcfg')

fd = os.open('/dev/uio/api', os.O_RDWR)
m = mmap.mmap(fileno=fd, length=mmap.PAGESIZE, offset=0x0)
gpio_array = np.recarray(1, regset_gpio, buf=m)
gpio = gpio_array[0]

GPIO T signal is inverted output enable, a native notation for Xilinx IOBUF Verilog primitives.
So to enable an output, the relevant bit in this register should be set to `0`.

The example then alternatively lights up `LED[3:0]` and `LED[7:4]` with a 0.2s delay between them.

In [ ]:
import time

# enable output
gpio.t = 0xffff00

# blink
for i in range(10):
    gpio.o = 0x0000f0
    time.sleep(0.2)
    gpio.o = 0x00000f
    time.sleep(0.2)

Reading from the input register should return the value on the pins.

In [ ]:
hex(gpio.i)

The next example provides an alternative mapping of the registers.
It is intended for testing 64 bit read access to registers.
Although the actual register is 32 bit the AXI interconnect
should split 64 bit transfers into two 32bit transfers.
This is usefull for example for 64 bit values like the Zynq DNA code.
Since the bus is little endian the high part of 64 bit data
is stored at address `0x0`, while the low part at address `0x4`.

In [ ]:
regset_test64 = np.dtype([
    ('ot', 'uint64'),
    ('i', 'uint32')
])

test64_array = np.recarray(1, regset_test64, buf=m)
test64 = test64_array[0]
gpio.t = 0x01234567
gpio.o = 0x89abcdef
hex(test64.ot)

Writing 64 bit data and reading it as two 32 bit registers should also work.

In [ ]:
test64.ot = 0xfedcba9876543210
hex(gpio.t), hex(gpio.o)